In [1]:
from nlpinitiative.data_preparation.data_management import DataManager
from nlpinitiative.modeling import train

from nlpinitiative.config import (
    PROCESSED_DATA_DIR
)

2025-03-19 17:07:45.764 | INFO     | nlpinitiative.config:<module>:11 - PROJ_ROOT path is: C:\Users\Daniel\Desktop\GitHub\NLPinitiative


# Data Preparation and Preprocessing:

In [2]:
dm = DataManager()
bin_data_obj, ml_data_obj = dm.prepare_and_preprocess_dataset('NLPinitiative_Master_Dataset.csv', PROCESSED_DATA_DIR)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7352 [00:00<?, ? examples/s]

Map:   0%|          | 0/3152 [00:00<?, ? examples/s]

Map:   0%|          | 0/7352 [00:00<?, ? examples/s]

Map:   0%|          | 0/3152 [00:00<?, ? examples/s]

Map:   0%|          | 0/7352 [00:00<?, ? examples/s]

Map:   0%|          | 0/3152 [00:00<?, ? examples/s]

# Loading/Creating model objects:

In [3]:
bin_class_model = train.get_bin_model()
ml_regress_model = train.get_ml_model()
bin_class_model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Initializing training arguments:

In [4]:
bin_targs = train.bin_train_args(num_train_epochs=3)
ml_targs = train.ml_regr_train_args(num_train_epochs=3)
bin_targs

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=500,
eval_strategy=IntervalStrategy.STEPS,
eval_u

# Initialize trainer objects with metrics:

In [5]:
bin_trainer = train.Trainer(
    model=bin_class_model,
    args=bin_targs,
    train_dataset=bin_data_obj.encoded_dataset['train'],
    eval_dataset=bin_data_obj.encoded_dataset['test'],
    compute_metrics=train.compute_bin_metrics
)

ml_trainer = train.RegressionTrainer(
    model=ml_regress_model,
    args=ml_targs,
    train_dataset=ml_data_obj.encoded_dataset['train'],
    eval_dataset=ml_data_obj.encoded_dataset['test'],
    compute_metrics=train.compute_reg_metrics
)

bin_trainer

# Training:

In [ ]:
bin_eval_res, ml_eval_res = train.train(bin_trainer=bin_trainer, ml_trainer=ml_trainer)

Step,Training Loss,Validation Loss


# Uploading Best Models:

In [ ]:
token = 'HF TOKEN GOES HERE'
train.upload_best_models(token=token)